In [34]:
import sys
sys.path.append('..')

import os
from sqlalchemy import create_engine, text
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
from shapely import wkb
from geopandas.tools import sjoin

from utilities_amigocloud import AmigocloudFunctions

In [35]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
PATH_CAT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\SHP\CATASTRO_S13.shp'
PATH_XLSX_GRUPOS = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'

In [36]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_planificacion():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_pulverizacion.planificacion_pulv
            WHERE procesado=false;
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def marcar_como_procesado(id_os):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.planificacion_pulv
                SET procesado = true
                WHERE id = :id_os
            """)
            conn.execute(query, {"id_os": id_os})
            print(f"✔️ id {id_os} marcado como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None

def obtener_parte_diario_por_os(ors):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_pulverizacion.parte_diario_pulv where os = {ors}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def convertir_a_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    return geometry

def convertir_a_wkb(polygon):
    wkb_data = wkb.dumps(polygon, hex=True)
    return wkb_data

def cargar_a_amigocloud(gdf):
    # repreyectar a WGS84
    gdf_pla_gral = gdf.to_crs(epsg=4326)
    # convertir poligonos a multipoligonos
    gdf_pla_gral['geom'] = gdf_pla_gral['geom'].apply(convertir_a_multipolygon)
    
    gdf_pla_gral['unidad_01'] = gdf_pla_gral['unidad_01'].astype(int)
    gdf_pla_gral['unidad_03'] = gdf_pla_gral['unidad_03'].astype(int)
    gdf_pla_gral['os'] = gdf_pla_gral['os'].astype(int)
    gdf_pla_gral['soca'] = gdf_pla_gral['soca'].astype(int)
    gdf_pla_gral['id'] = gdf_pla_gral['id'].astype(int)
    gdf_pla_gral['inst'] = gdf_pla_gral['inst'].astype(int)
    
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 35248
    for index, row in gdf_pla_gral.iterrows():
        wkb_hex = convertir_a_wkb(row['geom'])
        insert_sql = f"""
            INSERT INTO dataset_360912 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry)
            VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['os']}', ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
        """
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        print(f'Lote registrado en AmigoCloud')
    return None

In [37]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

In [38]:
gdf_cat = gpd.read_file(PATH_CAT)
gdf_plan = obtener_planificacion()
gdf_plan

,id,geom,codigo_canero,nombre_canero,obs,procesado
0,37,MULTIPOINT (501605.878 8095135.574),41594,AGROPECUARIA CAMPO DULCE S.R.L.,,False
1,38,"MULTIPOINT (478893.024 8088924.459, 478919.335...",30035,GUTIERREZ AGUILERA WIDEN GUILLER,None,False
2,39,"MULTIPOINT (468372.655 8077599.746, 468492.931...",42376,HURTADO PEREDO ANTONIO,None,False
3,40,"MULTIPOINT (431285.440 8090587.648, 431007.303...",12308,ROCA CHAVEZ MAURICIO DANIEL,None,False
4,41,"MULTIPOINT (466656.846 8099736.591, 466382.467...",2549,CAMARGO REQUENA MARCO ANTONIO,None,False
5,42,"MULTIPOINT (466942.501 8100582.280, 466782.760...",1180,BALCAZAR BALCAZAR JESUS E.,None,False
6,43,"MULTIPOINT (468363.258 8101721.141, 468489.172...",15284,ROJAS CORDOVA ROGELIO FELIPE,None,False
7,44,"MULTIPOINT (474539.606 8095637.819, 472600.629...",515,AGUILERA OLGA RIVERO VDA DE,None,False


# CARGAR PARTE DIARIO

In [39]:
gdf_intersect_result = gpd.sjoin(gdf_cat, gdf_plan, how='inner')
os_para_cargar = list(set(gdf_intersect_result['id_right']))
os_para_cargar

[37, 38, 39, 40, 41, 42, 43, 44]

In [40]:
gdf_result = gdf_intersect_result[['geometry', 'unidad_01', 'unidad_02', 'codigo_canero', 'nombre_canero', 'unidad_05', 'area', 'id_right', 'soca', 'zona']].copy()
gdf_result['area'] = gdf_result.geometry.area / 10000
gdf_result.rename(columns={
    'geometry': 'geom',
    'codigo_canero': 'unidad_03',
    'nombre_canero': 'unidad_04',
    'id_right': 'os',
    'zona': 'inst'
}, inplace=True)

gdf_result = gdf_result.set_geometry("geom")
gdf_result['unidad_01'] = gdf_result['unidad_01'].astype(int)
gdf_result['unidad_03'] = gdf_result['unidad_03'].astype(int)
gdf_result['os'] = gdf_result['os'].astype(int)
gdf_result['soca'] = gdf_result['soca'].astype(int)
gdf_result['inst'] = gdf_result['inst'].astype(int)

In [41]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst
1895,"POLYGON ((468236.075 8101411.439, 468661.490 8...",2271,SAN MARTIN 27,15284,ROJAS CORDOVA ROGELIO FELIPE,L1.1,4.547622,43,-1,50
2226,"POLYGON ((468661.490 8101577.284, 468236.075 8...",2271,SAN MARTIN 27,15284,ROJAS CORDOVA ROGELIO FELIPE,L1.2,22.717481,43,-1,50
2162,"POLYGON ((501210.863 8095048.211, 501490.109 8...",30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,ER-L18,43.679481,37,-1,50
2510,"POLYGON ((474681.586 8095879.501, 474908.347 8...",17,LA ESTACADA--OLGUITA,515,AGUILERA OLGA RIVERO VDA DE,A5.1,20.140508,44,0,50
3003,"POLYGON ((472793.293 8095627.527, 472849.118 8...",17,LA ESTACADA--OLGUITA,515,AGUILERA OLGA RIVERO VDA DE,C14.1,18.103557,44,0,50
9788,"POLYGON ((472814.137 8096450.894, 472815.410 8...",17,LA ESTACADA--OLGUITA,515,AGUILERA OLGA RIVERO VDA DE,C23.1,3.004324,44,0,50
10001,"POLYGON ((472436.354 8096702.851, 472713.849 8...",17,LA ESTACADA--OLGUITA,515,AGUILERA OLGA RIVERO VDA DE,C24,18.450238,44,0,50
10655,"POLYGON ((472982.261 8095881.153, 472988.181 8...",17,LA ESTACADA--OLGUITA,515,AGUILERA OLGA RIVERO VDA DE,C14.3,4.816453,44,1,50
5737,"POLYGON ((466918.645 8099789.384, 466913.595 8...",1816,LOS MAGUESES--BALCAZAR JESUS,1180,BALCAZAR BALCAZAR JESUS E.,L9,3.462846,42,1,69
6061,"POLYGON ((466923.915 8100348.638, 466847.112 8...",1816,LOS MAGUESES--BALCAZAR JESUS,1180,BALCAZAR BALCAZAR JESUS E.,L3,5.862404,42,2,69


In [42]:
# GARANTIZAR LA INST DEL CAÑERO
# cargar grupos de cosecha
df_grupos = pd.read_excel(PATH_XLSX_GRUPOS, sheet_name='CODIGOS')
df_grupos = df_grupos[df_grupos['INSTITUCION'].notna()]
df_grupos['CODIGO CAÑERO'] = df_grupos['CODIGO CAÑERO'].astype(int)
df_grupos['INSTITUCION'] = df_grupos['INSTITUCION'].astype(int)

# Crear un diccionario de mapeo: {codigo_cañero: institucion}
mapa_institucion = df_grupos.set_index('CODIGO CAÑERO')['INSTITUCION'].to_dict()
# Reemplazar los valores de 'inst' en gdf_result usando el diccionario
# a partir del diccionario, busca el codigo cañero (unidad_03), y reempalza la institucion
gdf_result['inst'] = gdf_result['unidad_03'].map(mapa_institucion)

In [43]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst
1895,"POLYGON ((468236.075 8101411.439, 468661.490 8...",2271,SAN MARTIN 27,15284,ROJAS CORDOVA ROGELIO FELIPE,L1.1,4.547622,43,-1,79
2226,"POLYGON ((468661.490 8101577.284, 468236.075 8...",2271,SAN MARTIN 27,15284,ROJAS CORDOVA ROGELIO FELIPE,L1.2,22.717481,43,-1,79
2162,"POLYGON ((501210.863 8095048.211, 501490.109 8...",30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,ER-L18,43.679481,37,-1,50
2510,"POLYGON ((474681.586 8095879.501, 474908.347 8...",17,LA ESTACADA--OLGUITA,515,AGUILERA OLGA RIVERO VDA DE,A5.1,20.140508,44,0,50
3003,"POLYGON ((472793.293 8095627.527, 472849.118 8...",17,LA ESTACADA--OLGUITA,515,AGUILERA OLGA RIVERO VDA DE,C14.1,18.103557,44,0,50
9788,"POLYGON ((472814.137 8096450.894, 472815.410 8...",17,LA ESTACADA--OLGUITA,515,AGUILERA OLGA RIVERO VDA DE,C23.1,3.004324,44,0,50
10001,"POLYGON ((472436.354 8096702.851, 472713.849 8...",17,LA ESTACADA--OLGUITA,515,AGUILERA OLGA RIVERO VDA DE,C24,18.450238,44,0,50
10655,"POLYGON ((472982.261 8095881.153, 472988.181 8...",17,LA ESTACADA--OLGUITA,515,AGUILERA OLGA RIVERO VDA DE,C14.3,4.816453,44,1,50
5737,"POLYGON ((466918.645 8099789.384, 466913.595 8...",1816,LOS MAGUESES--BALCAZAR JESUS,1180,BALCAZAR BALCAZAR JESUS E.,L9,3.462846,42,1,69
6061,"POLYGON ((466923.915 8100348.638, 466847.112 8...",1816,LOS MAGUESES--BALCAZAR JESUS,1180,BALCAZAR BALCAZAR JESUS E.,L3,5.862404,42,2,69


In [44]:
# agregar nuevos registros a la base de datos
gdf_result.to_postgis("parte_diario_pulv", obtener_engine(), schema="drones_pulverizacion", if_exists="append")

In [ ]:
os_para_cargar

In [45]:
# CARGAR A AMIGOCLOD
for i in os_para_cargar:
    plan_os = obtener_parte_diario_por_os(i)
    cargar_a_amigocloud(plan_os)
    marcar_como_procesado(i)

Lote registrado en AmigoCloud
✔️ id 37 marcado como procesado.
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 38 marcado como procesado.
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 39 marcado como procesado.
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 40 marcado como procesado.
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 41 marcado como procesado.
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoC